#**Importamos las librerias necesarias**

In [1]:
import numpy as np
import pandas as pd
import random

#**Importamos el dataset**

In [3]:
# Cargar los datos
df = pd.read_csv('equipos_futbol.csv')

# Simplificación: Generar una columna que indique el resultado (0 = derrota, 1 = empate, 2 = victoria)
# En este caso, usaremos una métrica como "%GanecomoLocal" para determinar el resultado.
df['Resultado'] = df['%GanecomoLocal'].apply(lambda x: 2 if x > 50 else (1 if x == 50 else 0))

#**Definimos los campos de entrenamiento**

In [4]:
# Definir las características (X) y la etiqueta de salida (y)
features = ['Posesion%', 'AciertoPase%', 'Goles', 'Tiros pp', 'Tarjetas Amarillas', 'Tarjetas Rojas']
X = df[features]
y = df['Resultado']

# Normalizar las características
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#**Definimos la tabla Q**

In [5]:
# Inicializar la tabla Q
n_states = len(X_scaled)  # Número de estados (cada partido es un estado)
n_actions = 3  # 0: derrota, 1: empate, 2: victoria
Q_table = np.zeros((n_states, n_actions))

# Parámetros de Q-learning
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.1  # Factor de exploración

#**Definimos funciones de elección**

In [6]:
# Función para elegir la acción (exploración-explotación)
def choose_action(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, n_actions - 1)  # Acción aleatoria (explorar)
    else:
        return np.argmax(Q_table[state, :])  # Mejor acción (explotar)

# Función para obtener la recompensa (predicción correcta o incorrecta)
def get_reward(predicted_action, true_result):
    return 1 if predicted_action == true_result else -1

#**Entrenamos el modelo**

In [7]:
# Entrenamiento del agente
n_episodes = 1000  # Número de episodios de entrenamiento

for episode in range(n_episodes):
    state = random.randint(0, n_states - 1)  # Estado inicial aleatorio (partido)

    for _ in range(100):  # Máximo 100 pasos por episodio
        # Elegir una acción (predicción)
        action = choose_action(state, epsilon)

        # Obtener el verdadero resultado del partido (etiqueta real)
        true_result = y.iloc[state]

        # Obtener la recompensa
        reward = get_reward(action, true_result)

        # Determinar el siguiente estado (partido siguiente)
        next_state = (state + 1) % n_states

        # Actualizar la tabla Q
        best_next_action = np.argmax(Q_table[next_state, :])
        Q_table[state, action] = Q_table[state, action] + alpha * (reward + gamma * Q_table[next_state, best_next_action] - Q_table[state, action])

        # Cambiar al siguiente estado
        state = next_state

#**Medimos la precision**

In [8]:
# Probar el agente después del entrenamiento
correct_predictions = 0
for state in range(n_states):
    action = np.argmax(Q_table[state, :])  # Predicción basada en la tabla Q
    true_result = y.iloc[state]
    if action == true_result:
        correct_predictions += 1

accuracy = correct_predictions / n_states
print(f"Precisión del agente con Q-learning: {accuracy * 100:.2f}%")

Precisión del agente con Q-learning: 100.00%
